In [ ]:
import requests
import json

# 1. Tải dữ liệu JSON hành chính Việt Nam
url = 'https://raw.githubusercontent.com/kenzouno1/DiaGioiHanhChinhVN/master/data.json'
response = requests.get(url)
data = response.json()

# 2. Lấy dữ liệu Thành phố Đà Nẵng
danang = next((city for city in data if city.get("Name") == "Thành phố Đà Nẵng"), None)
if not danang:
    raise ValueError("Không tìm thấy dữ liệu Thành phố Đà Nẵng")

# 3. Tạo cấu trúc dạng { "District": [Ward1, Ward2, ...] }
district_wards_map = {}
for district in danang.get("Districts", []):
    district_name = district.get("Name", "Không rõ quận/huyện")
    ward_names = [ward["Name"] for ward in district.get("Wards", []) if "Name" in ward]
    district_wards_map[district_name] = ward_names

# 4. Ghi ra file JSON
with open("danang_grouped_wards.json", "w", encoding="utf-8") as f:
    json.dump(district_wards_map, f, ensure_ascii=False, indent=2)

print("Đã tạo file danang_grouped_wards.json với dữ liệu dạng quận gồm các phường.")


In [1]:
import pandas as pd
import numpy as np
import json

# Đọc dữ liệu CSV
df = pd.read_csv("RawData.csv")

# Đọc dữ liệu mapping từ file JSON
with open("danang_grouped_wards.json", "r", encoding="utf-8") as f:
    district_ward_map = json.load(f)

# Tạo dict ward → district (đảo ngược lại để tra nhanh)
ward_to_district = {}
for district, wards in district_ward_map.items():
    for ward in wards:
        ward_to_district[ward] = district

# Xác định dòng thiếu District Name
mask_missing_district = df['District Name'].isna()

# Nếu thiếu District, cố gắng điền từ Ward Name
df.loc[mask_missing_district, 'District Name'] = df.loc[mask_missing_district, 'Ward Name'].map(ward_to_district)

# Kiểm tra kết quả
print(df[['Ward Name', 'District Name']][df['District Name'].isna()])  # các dòng vẫn chưa điền được

# Lưu lại nếu muốn
# df.to_csv("RawData_Fixed.csv", index=False)


                   Ward Name District Name
162           Phường Hòa Khê           NaN
190    Phường Hòa Thuận Đông           NaN
193           Phường Hòa Khê           NaN
194           Phường Hòa Khê           NaN
196        Phường An Hải Tây           NaN
...                      ...           ...
22878       Phường Nam Dương           NaN
22885     Phường An Hải Đông           NaN
22889     Phường Hải Châu  I           NaN
22890     Phường Hải Châu II           NaN
22899     Phường Hải Châu II           NaN

[487 rows x 2 columns]


C:\Users\DUC VAN\AppData\Local\Temp\ipykernel_10444\2440242367.py:6: DtypeWarning: Columns (3,5,8,31,32,34,35,37,39,40,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("RawData.csv")
